# PROJET 2 - Concevez une application au service de la santé publique

L'objectif de ce projet est de **concevoir une application au service de la santé publique**.

Nous devons utiliser la base de données OpenFoodFacts pour trouver des idées innovantes d’applications en lien avec l'alimentation.

OpenFoodFacts (https://world.openfoodfacts.org/) est une base de données citoyenne qui récupère des informations sur les produits de notre alimentation. Cette base de données contient différents champs sur : 
* les informations générales du produit
* un ensemble de tags
* des informations diverses
* les ingrédients du produit
* les informations nutritionnelles du produit

Ces données sont disponibles sur https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv





A partir des variables disponibles et utilisables pour l'application, nous pouvons élaborer une idée d'application, à savoir : 

* Evaluer la qualité des produits alimentaires pour les noter en se basant sur :
  * la qualité nutritionnelle du produit donnée par le Nutriscore
  * la quantité d'additifs
  * le nombre d'allergènes
  * le degré de transformation des produits donné par la classification NOVA

# **NETTOYAGE DES DONNEES**





# Importation des librairies

In [2]:
# coding: utf-8

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import timedelta, datetime
import scipy.stats as st
from collections import Counter
from pandas.api.types import CategoricalDtype

# Chargement des données

In [ ]:
# data = pd.read_csv('en.openfoodfacts.org.products.csv', sep = "\t", parse_dates=[3,6], low_memory=False)

Pour limiter la charge mémoire, on peut créer un dictionnaire des dtypes pour les colonnes qui ont des types mixtes lors du chargement des données

# Création d'un dictionnaire des dtypes 

In [3]:
## Création d'un dictionnaire avec les types des colonnes (0,8,13,22,23,27,28,29,31,52,64,38,55,132)

types_data = {'code': str, 'abbreviated_product_name' : str, 'packaging_text' : str, 'emb_codes' : str, 'emb_codes_tags' : str, 'manufacturing_places_tags' : str, 
              'manufacturing_places' : str, 'first_packaging_code_geo' : object, 'cities_tags' : str, 'allergens' : str , 'ingredients_from_palm_oil_tags' : str, 
              'ingredients_that_may_be_from_palm_oil_tags' : str, 'brand_owner' : str, 'alcohol_100g' : float}

In [4]:
data = pd.read_csv('en.openfoodfacts.org.products.csv', sep = "\t", parse_dates=[3,6], dtype = types_data)

In [ ]:
data.info()

# Propriétés générales du jeu de données data

In [ ]:
print(type(data))

In [ ]:
data.info()

In [ ]:
nb_data_intial = data.shape[0]
nb_data_avant = data.shape[0]

In [ ]:
def info_nettoyage(data):
  print("Le DataFrame est composé de", data.shape[0], "lignes et de", data.shape[1], "colonnes.")
  print(int(data.shape[0]/nb_data_intial * 100), '% jeu de données initial.')
  print(nb_data_intial - data.shape[0], 'lignes ont été supprimées au total.')
  print(nb_data_avant - data.shape[0], 'lignes ont été supprimées suite à ce nettoyage.')
  nb_donnees_tot = len(data)*len(data.columns)
  donnees_NaN = data.isnull().sum().sum()
  donnees_presentes = (nb_donnees_tot - donnees_NaN)/nb_donnees_tot
  print('Le DataFrame contient',  int(donnees_presentes * 100), '% de données présentes.')

In [ ]:
info_nettoyage(data)

In [ ]:
print("nombre de dimensions de data: ", data.ndim)
print("forme de data: ", data.shape)
print("taille de data: ", data.size)

In [ ]:
data.head(5) 

Nous avons à notre disposition un jeu de données de 1658499 lignes et de 184 colonnes.

In [ ]:
## Graphe du nombre de données manquantes

def graphe_donnees_manquantes(data) :
  nb_donnees_tot = len(data)*len(data.columns)
  donnees_NaN = data.isnull().sum().sum()
  donnees_presentes = nb_donnees_tot - donnees_NaN
  
  x = np.array([donnees_NaN, donnees_presentes])
  label = ["Données manquantes", "Données présentes"]
  
  plt.pie(x, explode=(0.1, 0), labels=label, colors=['tomato', 'blue'], autopct='%1.1f%%', shadow=True, startangle=90)
  plt.title('Diagramme circulaire des données présentes et manquantes')
  plt.show()

In [ ]:
graphe_donnees_manquantes(data)

# Suppression des colonnes vides

In [ ]:
cols_vides = [col for col in data.columns if data[col].isnull().all()]
cols_vides

In [ ]:
data.drop(cols_vides,axis=1,inplace=True)

In [ ]:
data = data.dropna(how = 'all')

In [ ]:
info_nettoyage(data)

In [ ]:
graphe_donnees_manquantes(data)

# Suppression des lignes avec des erreurs dans les informations nutritionnelles

In [ ]:
list(data.columns)

## Ingrédients >100 ou <0

In [ ]:
colonnes_compo_100g = [x for x in list(data.columns) if (x.endswith('_100g') and x not in (['energy-kj_100g', 'energy-kcal_100g', 'energy_100g', 'energy-from-fat_100g', 'nutrition-score-fr_100g', 'nutrition_score_uk_100g', 'ph_100g', 'glycemic-index_100g']))]
colonnes_compo_100g

In [ ]:
data[colonnes_compo_100g] = data[colonnes_compo_100g].astype(float)

In [ ]:
## Supprimer les lignes où une donnée dans les colonnes des informations nutritionnelles est négative et est supérieure à 100 (ici garder toutes les lignes où la valeur n'est pas inférieure à 0)

data = data[(np.invert(data[colonnes_compo_100g] > 100)).all(axis=1)]
data = data[(np.invert(data[colonnes_compo_100g] < 0)).all(axis=1)]

In [ ]:
data[colonnes_compo_100g].describe(include='all')

In [ ]:
info_nettoyage(data)

In [ ]:
graphe_donnees_manquantes(data)

## pH non compris entre 1 et 14

In [ ]:
data['ph_100g'] = data['ph_100g'].astype(float)
nb_data_avant = data.shape[0]

In [ ]:
data = data[(np.invert(data[['ph_100g']] < 1)).all(axis=1)]
data = data[(np.invert(data[['ph_100g']] > 14)).all(axis=1)]

In [ ]:
info_nettoyage(data)

In [ ]:
graphe_donnees_manquantes(data)

## Energie et indice de glycémie < 0

In [ ]:
nb_data_avant = data.shape[0]

In [ ]:
var_energie = ['energy-kj_100g', 'energy-kcal_100g', 'energy_100g', 'energy-from-fat_100g']
data = data[(np.invert(data[var_energie] < 0)).all(axis=1)]

In [ ]:
data = data[(np.invert(data[['glycemic-index_100g']] < 0)).all(axis=1)]
data = data[(np.invert(data[['glycemic-index_100g']] > 100)).all(axis=1)]

In [ ]:
info_nettoyage(data)

In [ ]:
graphe_donnees_manquantes(data)

## Ingrédients dont la somme totale >100

### Choix des variables 

La composition en nutriments de chaque produit est basée sur 100g. On peut donc supprimer les produits dont la somme totale des nutriments dépasse cette limite. 

Cependant, plusieurs nutriments se confondent dans la composition des produits tels que les glucides avec les sucres ou les lipides avec les acides gras saturés.

On doit donc choisir entre des groupes de nutriments quelle variable utilisée.

In [ ]:
acides_gras = ['fat_100g',  'saturated-fat_100g', 'monounsaturated-fat_100g', 'polyunsaturated-fat_100g',  'omega-3-fat_100g', 'omega-6-fat_100g', 'omega-9-fat_100g', 'trans-fat_100g', 'cholesterol_100g']
glucides = ['carbohydrates_100g','sugars_100g', '-sucrose_100g', '-glucose_100g', '-fructose_100g', '-lactose_100g', '-maltose_100g', '-maltodextrins_100g', 'starch_100g', 'polyols_100g']
sel = ['salt_100g', 'sodium_100g']
fibres = ['fiber_100g', '-soluble-fiber_100g', '-insoluble-fiber_100g']
proteines = ['proteins_100g', 'casein_100g', 'serum-proteins_100g']
fruits_legumes = ['fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-dried_100g', 'fruits-vegetables-nuts-estimate_100g']

Pour éviter de surestimer la somme totale des nutriments, on prend en compte les nutriments utilisés dans le Nutriscore qui est basé sur :
* l’énergie, les acides gras saturés, les sucres simples et le sel
* les fibres, les protéines, les fruits et légumes, les légumineuses et fruits à coque

Pour savoir parmi toutes ces variables lesquelles sont impliquées dans le calcul du Nutriscore, on peut faire un sous-échantillon du jeu de données et recalculer un Nutriscore avec les variables à choisir pour la somme totale

In [ ]:
## Attribution des points en fonction de la valeur du nutriment 

def energie(x, df):
  if x <= 335:
    E = 0
  else :
    E = df.loc[df['energie']==max((df[df['energie'] < x])['energie'])].index[0]
  return E

def sucres(x, df):
  if x == 0:
    S = 0
  else :
    S = df.loc[df['sucres']==max((df[df['sucres'] < x])['sucres'])].index[0] + 1
  return S

def graisses(x, df):
  if x <= 1:
    G = 0
  else :
    G = df.loc[df['graisses']==max((df[df['graisses'] < x])['graisses'])].index[0]
  return G

def sodium(x, df):
  if x <= 90:
    So = 0
  else :
    So = df.loc[df['sodium']==max((df[df['sodium'] < x])['sodium'])].index[0]
  return So

def fruits_legumes(x, df):
  if x <= 40:
    FL = 0
  else :
    FL = df.loc[df['fruits_legumes']==max((df[df['fruits_legumes'] < x])['fruits_legumes'])].index[0]
  return FL

def fibres(x, df):
  if x <= 0.7:
    F = 0
  else :
    F = df.loc[df['fibres']==max((df[df['fibres'] < x])['fibres'])].index[0]
  return F

def proteines(x, df):
  if x <= 1.6:
    Pr = 0
  else :
    Pr = df.loc[df['proteines']==max((df[df['proteines'] < x])['proteines'])].index[0]
  return Pr

In [ ]:
## Tableau des différents seuils des facteurs nutritionnels

df = pd.DataFrame({'energie': [0, 335, 670, 1005, 1340, 1675, 2010, 2345, 2680, 3015, 3350], 
                   'sucres': [0, 1.5, 3, 4.5, 6, 7.5, 9, 10.5, 12, 13.5, 100],
                   'graisses' : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                   'sodium' : [0, 90, 180, 270, 360, 450, 540, 630, 720, 810, 900],
                   'fruits_legumes' : [0, 40, 60, np.nan, np.nan, 80, np.nan, np.nan, np.nan, np.nan, np.nan], 
                   'fibres' : [0, 0.7, 1.4, 2.1, 2.8, 3.5, np.nan, np.nan, np.nan, np.nan, np.nan],
                   'proteines' : [0, 1.6, 3.2, 4.8, 6.4, 8, np.nan, np.nan, np.nan, np.nan, np.nan]})

In [ ]:
## Calcul du Nutri-score à partir de la valeur des nutriments

def nutri(subset):
  Nutri = []
  for i in range(len(subset)):
    x = list(subset.iloc[i]) ## subset dans l'ordre energie, sucres, graisses saturées, sodium
    N = energie(x[1], df) + graisses(x[2], df) + sucres(x[3], df) + sodium(x[4], df)
    P = fibres(x[5], df) + proteines(x[6], df) + fruits_legumes(x[7], df)
    if N >= 11:
      if fruits_legumes(x[7], df) == 5:
        Nutri.append(N - P)
      else :
        Nutri.append(N - (fibres(x[5], df) + fruits_legumes(x[7], df)))
    else:
      Nutri.append(N - P)
  return Nutri

D'après le calcul du Nutriscore (https://world.openfoodfacts.org/nutriscore), les boisssons et le fromage sont des cas particuliers lors du calcul. Pour plus de facilité dans le calcul, on n'utilise qu'un échantillon sans donnée manquante ne comportant ni fromage, ni boisson.

In [ ]:
## Cas 1 : Entre 'saturated-fat_100g' et 'fat_100g'

subset1 = data[['nutrition-score-fr_100g', 'energy-kj_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g','fiber_100g', 'proteins_100g', 'fruits-vegetables-nuts-estimate_100g', 'pnns_groups_1', 'pnns_groups_2']].dropna()
subset1 = subset1[(np.invert(subset1[['pnns_groups_1']] == 'Beverages')).all(axis=1)]
subset1 = subset1[(np.invert(subset1[['pnns_groups_1']] == 'unknown')).all(axis=1)]
subset1 = subset1[(np.invert(subset1[['pnns_groups_2']] == 'Cheese')).all(axis=1)]
subset1['nutri_cal'] = nutri(subset1)
subset1[["nutrition-score-fr_100g", "nutri_cal"]] = subset1[["nutrition-score-fr_100g", "nutri_cal"]].astype('int64')
print('Cas 1')
print('Avec saturated-fat_100g :',st.pearsonr(subset1['nutrition-score-fr_100g'],subset1['nutri_cal'])[0])

subset = data[['nutrition-score-fr_100g', 'energy-kj_100g', 'fat_100g', 'sugars_100g', 'salt_100g','fiber_100g', 'proteins_100g', 'fruits-vegetables-nuts-estimate_100g', 'pnns_groups_1', 'pnns_groups_2']].dropna()
subset = subset[(np.invert(subset[['pnns_groups_1']] == 'Beverages')).all(axis=1)]
subset = subset[(np.invert(subset[['pnns_groups_1']] == 'unknown')).all(axis=1)]
subset = subset[(np.invert(subset[['pnns_groups_2']] == 'Cheese')).all(axis=1)]
subset['nutri_cal'] = nutri(subset)
subset[["nutrition-score-fr_100g", "nutri_cal"]] = subset[["nutrition-score-fr_100g", "nutri_cal"]].astype('int64')
print('Avec fat_100g :',st.pearsonr(subset['nutrition-score-fr_100g'],subset['nutri_cal'])[0])

## Cas 2 : 'sugars_100g' et 'carbohydrates_100g'

subset = data[['nutrition-score-fr_100g', 'energy-kj_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'salt_100g','fiber_100g', 'proteins_100g', 'fruits-vegetables-nuts-estimate_100g', 'pnns_groups_1', 'pnns_groups_2']].dropna()
subset = subset[(np.invert(subset[['pnns_groups_1']] == 'Beverages')).all(axis=1)]
subset = subset[(np.invert(subset[['pnns_groups_1']] == 'unknown')).all(axis=1)]
subset = subset[(np.invert(subset[['pnns_groups_2']] == 'Cheese')).all(axis=1)]
subset['nutri_cal'] = nutri(subset)
subset[["nutrition-score-fr_100g", "nutri_cal"]] = subset[["nutrition-score-fr_100g", "nutri_cal"]].astype('int64')
print('Cas 2')
print('Avec carbohydrates_100g :',st.pearsonr(subset['nutrition-score-fr_100g'],subset['nutri_cal'])[0])

## Cas 3 : 'salt_100g' et 'sodium_100g'

subset = data[['nutrition-score-fr_100g', 'energy-kj_100g', 'saturated-fat_100g', 'sugars_100g', 'sodium_100g','fiber_100g', 'proteins_100g', 'fruits-vegetables-nuts-estimate_100g', 'pnns_groups_1', 'pnns_groups_2']].dropna()
subset = subset[(np.invert(subset[['pnns_groups_1']] == 'Beverages')).all(axis=1)]
subset = subset[(np.invert(subset[['pnns_groups_1']] == 'unknown')).all(axis=1)]
subset = subset[(np.invert(subset[['pnns_groups_2']] == 'Cheese')).all(axis=1)]
subset['nutri_cal'] = nutri(subset)
subset[["nutrition-score-fr_100g", "nutri_cal"]] = subset[["nutrition-score-fr_100g", "nutri_cal"]].astype('int64')
print('Cas 3')
print('Avec sodium_100g :',st.pearsonr(subset['nutrition-score-fr_100g'],subset['nutri_cal'])[0])

## Cas 4 : 'fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-dried_100g' et 'fruits-vegetables-nuts-estimate_100g']

subset = data[['nutrition-score-fr_100g', 'energy-kj_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g','fiber_100g', 'proteins_100g', 'fruits-vegetables-nuts_100g', 'pnns_groups_1', 'pnns_groups_2']].dropna()
subset = subset[(np.invert(subset[['pnns_groups_1']] == 'Beverages')).all(axis=1)]
subset = subset[(np.invert(subset[['pnns_groups_1']] == 'unknown')).all(axis=1)]
subset = subset[(np.invert(subset[['pnns_groups_2']] == 'Cheese')).all(axis=1)]
subset['nutri_cal'] = nutri(subset)
subset[["nutrition-score-fr_100g", "nutri_cal"]] = subset[["nutrition-score-fr_100g", "nutri_cal"]].astype('int64')
print('Cas 4')
print('Avec fruits-vegetables-nuts_100g :',st.pearsonr(subset['nutrition-score-fr_100g'],subset['nutri_cal'])[0])

subset = data[['nutrition-score-fr_100g', 'energy-kj_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g','fiber_100g', 'proteins_100g', 'fruits-vegetables-nuts-dried_100g', 'pnns_groups_1', 'pnns_groups_2']].dropna()
subset = subset[(np.invert(subset[['pnns_groups_1']] == 'Beverages')).all(axis=1)]
subset = subset[(np.invert(subset[['pnns_groups_1']] == 'unknown')).all(axis=1)]
subset = subset[(np.invert(subset[['pnns_groups_2']] == 'Cheese')).all(axis=1)]
subset['nutri_cal'] = nutri(subset)
subset[["nutrition-score-fr_100g", "nutri_cal"]] = subset[["nutrition-score-fr_100g", "nutri_cal"]].astype('int64')
print('Avec fruits-vegetables-nuts-dried_100g :',st.pearsonr(subset['nutrition-score-fr_100g'],subset['nutri_cal'])[0])

D'après les résultats du calcul du coefficient de Pearson, la plus forte corrélation se fait avec les variables : 'saturated-fat_100g', 'sugars_100g', 'salt_100g','fiber_100g', 'proteins_100g', 'fruits-vegetables-nuts-estimate_100g'



In [ ]:
print(subset1.columns)

In [ ]:
plt.figure(1,figsize=(15,10))
plt.gcf().subplots_adjust(left = 0.125, bottom = 0.2, right = 1.5,
                          top = 0.9, wspace = 0.2, hspace = 0.6)

plt.subplot(1,2,1)
subset1["nutrition-score-fr_100g"].hist(legend="nutrition-score-fr_100g")

plt.subplot(1,2,2)
subset1["nutri_cal"].hist(legend="nutri_cal")

plt.show()

In [ ]:
st.pearsonr(subset1['nutrition-score-fr_100g'],subset1['nutri_cal'])[0]

On obtient un coefficient de Pearson proche de 0.9, ce qui laisse à pense qu'il y a effectivement une corrélation entre ces 2 variables (comme ce que l'on peut observer sur les 2 histogrammes).

In [ ]:
taille_classe = 4
groupes = []

subset1[["nutrition-score-fr_100g", "nutri_cal"]] = subset1[["nutrition-score-fr_100g", "nutri_cal"]].astype(float)

tranches = np.arange(0, max(subset1["nutrition-score-fr_100g"]), taille_classe)
tranches += taille_classe/2
indices = np.digitize(subset1["nutrition-score-fr_100g"], tranches) 

for ind, tr in enumerate(tranches):
    nutri_cal = subset1.loc[indices==ind,"nutri_cal"] 
    if len(nutri_cal) > 0:
        g = {
            'valeurs': nutri_cal,
            'centre_classe': tr-(taille_classe/2),
            'taille': len(nutri_cal),
            'quartiles': [np.percentile(nutri_cal,p) for p in [25,50,75]]
        }
        groupes.append(g)

# affichage des boxplots
plt.boxplot([g["valeurs"] for g in groupes],
            positions= [g["centre_classe"] for g in groupes], 
            showfliers= False,
            widths= taille_classe*0.7,
)
plt.xlabel("nutrition-score-fr_100g")
plt.ylabel("nutri_cal")
plt.title("Boxplot des agrégats")

# affichage des effectifs de chaque classe
for g in groupes:
    plt.text(g["centre_classe"],0,"(n={})".format(g["taille"]),horizontalalignment='center',verticalalignment='top')     
plt.show()

# affichage des quartiles
for n_quartile in range(3):
    plt.plot([g["centre_classe"] for g in groupes],
             [g["quartiles"][n_quartile] for g in groupes])
plt.xlabel("nutrition-score-fr_100g")
plt.ylabel("nutri_cal")
plt.title("Plot des agrégats")
plt.show()

In [ ]:
del subset
del subset1

### Suppression des lignes

In [ ]:
nutriments = ['saturated-fat_100g', 'sugars_100g', 'salt_100g', 'fiber_100g', 'proteins_100g','fruits-vegetables-nuts-estimate_100g']

In [ ]:
df_small = data[nutriments]
df_small = df_small.dropna(how = 'all')
correlation_mat = df_small.corr()
mask = np.zeros_like(correlation_mat)
mask[np.triu_indices_from(mask)] = True
plt. figure(figsize = (20, 10)) 
sns.heatmap(correlation_mat, mask = mask, annot = True)
plt.title('Matrice des corrélations')
plt.show()

Ces variables sont très peu corrélées (plus grande corrélation à -0.3 entre 'fruits-vegetables-nuts-estimate_100g' et 'saturated-fat_100g')

In [ ]:
data[nutriments].describe(include='all')

In [ ]:
data['tot_compo'] = data[nutriments].sum(axis = 1)
data.head(5)

In [ ]:
nb_data_avant = data.shape[0]

In [ ]:
## Si somme colonnes > 100 -> supprimer ligne
data.drop(data[data['tot_compo'] >100 ].index, axis=0, inplace=True) # supprimer lignes dont la composition dépasse 100g

In [ ]:
data.drop(['tot_compo'], axis = 1, inplace = True)

In [ ]:
data = data.dropna(how = 'all') 

In [ ]:
info_nettoyage(data)

# Choix entre countries, countries_tags et countries_en

In [ ]:
nb_data_avant = data.shape[0]

In [ ]:
data[['countries', 'countries_tags', 'countries_en']].head(12)

In [ ]:
data['countries'].unique()

In [ ]:
data['countries_tags'].unique()

In [ ]:
data['countries_en'].unique()

Les variables 'countries' et 'countries_en' ne sont pas mises en forme. On ne garde que la variable 'countries_tags'

In [ ]:
data.drop(['countries_en'], axis = 1, inplace = True)
data.drop(['countries'], axis = 1, inplace = True)

## Sélection du pays d'étude

In [ ]:
data['countries_tags'] = data['countries_tags'].astype('category')

In [ ]:
len(data['countries_tags'].unique())

In [ ]:
data['countries_tags'].count()

In [ ]:
def most_common_words(labels):
    words = []
    for lab in labels:
        words += lab.split(",")
    counter = Counter(words)
    for word in counter.most_common():
        print(word)

most_common_words(data['countries_tags'].dropna().values)

Les produits venant de France sont les plus représentatifs dans le DataFrame.

La France apparait 761 141 fois, soit plus du double du 2e pays qui sont les Etats-Unis qui apparaissent 346 703 fois.

On sélectionne pour l'application uniquement les produits vendus en France.

In [ ]:
liste_pays = ['en:france', 'fr:francia', 'en:francia', 'en:frankreich', 'en:frankreich', 'en:france-francais', 'fr:frankreich', 'fr:frankrijk', 'fr:franca', 'ca:franca', 
              'de:francia', 'pt:francia', 'en:frankrijk', 'fr:francja', 'es:franca', 'en:francja', 'it:frankreich', 'en:franța', 'es:frankrijk', 'fr:francie', 'fr:γαλλία', 
              'fr:فرنسا', 'fr:frankrike', 'de:francja', 'en:franca', 'fr:francija', 'en:franciaorszag', 'es:fransa']

pattern = '|'.join(liste_pays)

data1 = data.loc[data['countries_tags'].str.contains(pattern, na=False, case=False)]

In [ ]:
data1.drop(['countries_tags'], axis = 1, inplace = True)

In [ ]:
data1 = data1.dropna(how = 'all')

In [ ]:
info_nettoyage(data1)

# Choix des variables

In [ ]:
nb_data_avant_1 = data1.shape[0]

Dans ce jeu de données, on a beaucoup de variables (172 colonnes).

Une première sélection est de traiter les colonnes utiles qui se rapportent à la même information. Il faut donc choisir entre les variables en double parmi les variables utiles pour l'application (les variables non utilisées ne seront pas traitées).

## Sélection des variables utiles pour l'application

In [ ]:
list(data1.columns)

On sélectionne les variables pertinentes et utiles pour l'idée d'application présentée : 
* informations générales = 'code', 'url', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name'
* informations sur la qualité nutritionnelle = 'nutriscore_grade', 'nutrition-score-fr_100g', 'nutriscore_score'
* colonnes finissant par '_100g'
* informations sur le produit = 'additives_n', 'allergens', 'nova_group', 'pnns_groups_1', 'pnns_groups_2',

Cette liste peut être ammenée à changer

In [ ]:
## sélectionner toutes les variables dans tous les formats qui répondent au projet, qui sont pertinentes au projet

colonnes_utiles = ['code', 'url', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 
                   'nutriscore_grade', 'nutrition-score-fr_100g', 'nutriscore_score',
                   'additives_n', 
                   'allergens'
                   'nova_group', 
                   'pnns_groups_1', 'pnns_groups_2']

toutes_colonnes_compo_100g = [x for x in list(data1.columns) if (x.endswith('_100g') and x not in (['nutrition-score-fr_100g', 'nutrition_score_uk_100g']))]

colonnes_utiles = colonnes_utiles + toutes_colonnes_compo_100g

## Types des variables

In [ ]:
data1['created_datetime'] = data1['created_datetime'].astype('datetime64[ns]')

var_int = ['additives_n', 'nutriscore_score', 'nutrition-score-fr_100g', 'created_t', 'last_modified_t']
data1[var_int] = data1[var_int].astype(np.float).astype('Int32')

var_cat = ['nova_group', 'pnns_groups_1', 'pnns_groups_2', 'nutriscore_grade']
data1[var_cat] = data1[var_cat].astype('category')

## Variables temporelles

In [ ]:
nb_data_avant = data1.shape[0]

In [ ]:
var_temp = ['created_t','created_datetime','last_modified_t', 'last_modified_datetime']

In [ ]:
plt.figure(1,figsize=(10,12))
plt.gcf().subplots_adjust(left = 0.125, bottom = 0.2, right = 1.5,
                          top = 0.9, wspace = 0.5, hspace = 0.6)

plt.subplot(2,2,1)
data1["last_modified_t"].hist(legend="last_modified_t")

plt.subplot(2,2,2)
data1["last_modified_datetime"].hist(density=True, legend="last_modified_datetime")

plt.subplot(2,2,3)
data1["created_t"].hist(legend="created_t")

plt.subplot(2,2,4)
data1["created_datetime"].hist(density=True, legend="last_modified_t")

plt.show()

In [ ]:
## Création d'une nouvelle variable de 'last_modified_datetime' au format d'horodatage UNIX (nombre de secondes depuis le 1er janvier 1970)
data1['last_modified_datetime_ns'] =  (pd.to_datetime(data1['last_modified_datetime']) - pd.Timestamp("1970-01-01 00:00:00+00:00")) // pd.Timedelta('1ns') 

In [ ]:
X = 'last_modified_datetime_ns'
Y = 'last_modified_t'
print("La corrélation de Pearson est de :", st.pearsonr(data1[X],data1[Y])[0])

Le coefficient de Pearson est de 1, ces 2 variables sont donc corrélées.

On ne garde qu'une seule variable temporelle qui est utile : 'last_modified_t'

In [ ]:
data1.drop(['created_t','created_datetime', 'last_modified_datetime', 'last_modified_datetime_ns'], axis = 1, inplace = True)

In [ ]:
data1 = data1.dropna(how = 'all')

In [ ]:
info_nettoyage(data1)

## Choix entre nutriscore_score et nutrition-score-fr_100g



In [ ]:
nb_data_avant = data1.shape[0]

In [ ]:
var_nutri_quanti = ['nutriscore_score', 'nutrition-score-fr_100g']

In [ ]:
data1[var_nutri_quanti].dropna()

In [ ]:
plt.figure(1,figsize=(10,5))
plt.gcf().subplots_adjust(left = 0.125, bottom = 0.2, right = 1.5,
                          top = 0.9, wspace = 0.5, hspace = 0.6)

plt.subplot(1,2,1)
data1["nutriscore_score"].hist(legend="nutriscore_score")

plt.subplot(1,2,2)
data1["nutrition-score-fr_100g"].hist(legend="nutrition-score-fr_100g")

plt.show()

In [ ]:
print(data1['nutriscore_score'].isnull().sum(axis = 0))
print(data1['nutrition-score-fr_100g'].isnull().sum(axis = 0))

In [ ]:
data1[var_nutri_quanti].describe(include = 'all')

In [ ]:
tab = data1[var_nutri_quanti].dropna(how = 'all')
tab[var_nutri_quanti].astype('Int32')
sum(list((tab['nutriscore_score'] - tab['nutrition-score-fr_100g'])**2))

Il n'y a pas de différence entre ces 2 variables. On garde une seule variable pour la suite des analyses, soit : 'nutrition-score-fr_100g'

In [ ]:
data1.drop(['nutriscore_score'], axis = 1, inplace = True)

In [ ]:
info_nettoyage(data1)

## Choix entre 'energy_100g' et 'energy-kj_100g'

In [ ]:
nb_data_avant = data1.shape[0]

In [ ]:
var_energie = ['energy_100g', 'energy-kj_100g']

In [ ]:
tab = data1[var_energie].dropna()
tab = tab.dropna(how = 'any')
tab[var_energie] = tab[var_energie].astype(int)
tab[var_energie].isnull().sum(axis = 0)

In [ ]:
sum(list((tab['energy_100g'] - tab['energy-kj_100g'])**2))

Il n'y a pas de différence entre ces 2 variables. On ne garde que la variable 'energy_100g' qui est la plus remplie.

In [ ]:
data1.drop(['energy-kj_100g'], axis = 1, inplace = True)

In [ ]:
data1 = data1.dropna(how = 'all')

In [ ]:
info_nettoyage(data1)

## Description du choix des variables utiles pour l'application

In [ ]:
## Nettoyage total des colonnes en double
nb_data_avant = nb_data_avant_1
info_nettoyage(data1)

In [ ]:
graphe_donnees_manquantes(data1)

# Nettoyage des colonnes 'pnns_groups_1' et 'pnns_groups_2'

In [ ]:
nb_data_avant = data1.shape[0]

In [ ]:
data1['pnns_groups_1'].value_counts()

In [ ]:
data1['pnns_groups_1'] = data1["pnns_groups_1"].replace(['unknown'],np.nan)
data1['pnns_groups_1'] = data1["pnns_groups_1"].replace(['sugary-snacks'],'Sugary snacks')
data1['pnns_groups_1'] = data1["pnns_groups_1"].replace(['salty-snacks'],'Salty snacks')
data1['pnns_groups_1'] = data1["pnns_groups_1"].replace(['fruits-and-vegetables'],'Fruits and vegetables')
data1['pnns_groups_1'] = data1["pnns_groups_1"].replace(['cereals-and-potatoes'],'Cereals and potatoes')

In [ ]:
data1['pnns_groups_1'].value_counts()

In [ ]:
data1['pnns_groups_2'].value_counts()

In [ ]:
data1['pnns_groups_2'] = data1["pnns_groups_2"].replace(['unknown'],np.nan)
data1['pnns_groups_2'] = data1["pnns_groups_2"].replace(['vegetables'],'Vegetables')
data1['pnns_groups_2'] = data1["pnns_groups_2"].replace(['legumes'],'Legumes')
data1['pnns_groups_2'] = data1["pnns_groups_2"].replace(['nuts'],'Nuts')
data1['pnns_groups_2'] = data1["pnns_groups_2"].replace(['cereals'],'Cereals')
data1['pnns_groups_2'] = data1["pnns_groups_2"].replace(['fruits'],'Fruits')
data1['pnns_groups_2'] = data1["pnns_groups_2"].replace(['Pizza pies and quiches'],'Pizza pies and quiche')

In [ ]:
data1['pnns_groups_2'].value_counts()

In [ ]:
print(data1['pnns_groups_1'].isnull().sum(axis = 0))

In [ ]:
print(data1['pnns_groups_2'].isnull().sum(axis = 0))

In [ ]:
data1[['pnns_groups_1', 'pnns_groups_2']].dropna(how='all').isnull().sum(axis = 0)

On utilise la colonne 'pnns_groups_2' pour remplir la colonne 'pnns_groups_1' en créant un dictionnaire des catégories entre 'pnns_groups_1' et 'pnns_groups_2'



## Imputation de 'pnns_groups_1' par les données de 'pnns_groups_2'

In [ ]:
sns.factorplot(x = "pnns_groups_1", hue = "pnns_groups_2", data = data1[['pnns_groups_1', 'pnns_groups_2']], kind = "count", linewidth = 15, size=12, aspect=2, palette=sns.color_palette("tab10"))

In [ ]:
## Dictionnaire des catégories

dicti = {'Sugary snacks' : ['Sweets',' Biscuits and cakes', 'Chocolate products', 'pastries'],
         'Fish Meat Eggs' : ['Meat', 'Fish and seafood', 'Eggs', 'Offals', 'Processed meat'],
         'Milk and dairy products' : ['Cheese', 'Milk and yogurt',' Dairy desserts', 'Ice cream'],
         'Cereals and potatoes': ['Cereals', 'Potatoes', 'Bread', 'Breakfast cereals'],
         'Fat and sauces': ['Dressings and sauces', 'Fats'],
         'Beverages': ['Sweetened beverages',' Unsweetened beverages', 'Alcoholic beverages', 'Artificially sweetened beverages', 'Waters and flavored waters', 'Teas and herbal teas and coffees', 'Plant-based milk substitutes', 'Fruit juices', 'Fruit nectars'],
         'Fruits and vegetables': ['Vegetables', 'Fruits', 'Legumes', 'Nuts', 'Dried fruits'],
         'Composite foods': ['Pizza pies and quiche', 'Soups', 'Sandwiches', 'One-dish meals'],
         'Salty snacks': ['Appetizers', 'Salty and fatty products']}

In [ ]:
data1['pnns_groups_1_bis'] = data1['pnns_groups_1']

In [ ]:
data1['pnns_groups_1_bis'].value_counts()

In [ ]:
print(data1[['pnns_groups_1', 'pnns_groups_1_bis','pnns_groups_2']].dropna(how='all').isnull().sum(axis = 0))

In [ ]:
data1['pnns_groups_1_bis'] = np.where(data1['pnns_groups_1_bis'].isna(), data1['pnns_groups_2'],  data1['pnns_groups_1'])

In [ ]:
data1['pnns_groups_1_bis'].value_counts()

In [ ]:
print(data1[['pnns_groups_1', 'pnns_groups_1_bis','pnns_groups_2']].dropna(how='all').isnull().sum(axis = 0))

In [ ]:
for i in range(len(list(dicti.keys()))-1):
  data1['pnns_groups_1_bis'] = data1['pnns_groups_1_bis'].replace(dicti[list(dicti.keys())[i]],list(dicti.keys())[i])

In [ ]:
data1['pnns_groups_1_bis'].value_counts()

In [ ]:
data1['pnns_groups_1_bis'] = data1['pnns_groups_1_bis'].astype('category')

In [ ]:
plt.figure(1,figsize=(30,20))
plt.gcf().subplots_adjust(left = 0.125, bottom = 0.2, right = 1.5,
                          top = 0.9, wspace = 0.1, hspace = 0.6)

plt.subplot(1,2,1)
data1["pnns_groups_1_bis"].hist()
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.xticks(rotation=45)
plt.title('Histogramme de pnns_groups_1_bis', fontsize=30)

plt.subplot(1,2,2)
data1["pnns_groups_1"].hist()
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.xticks(rotation=45)
plt.title('Histogramme de pnns_groups_1', fontsize=30)
plt.show()

In [ ]:
data1[["pnns_groups_1", "pnns_groups_1_bis"]].describe(include='all')

Ces 2 variables ont la même distribution et les mêmes statistiques descriptives (pas de grandes modifications sur la distribution des données suite à l'imputation)

In [ ]:
data1.drop(['pnns_groups_1'], axis = 1, inplace = True)

In [ ]:
info_nettoyage(data1)

# Supprimer les lignes sans les informations utiles

In [ ]:
data1 = data1.dropna(how = 'all')

In [ ]:
nb_data_avant = data1.shape[0]

Notre idée d'application est basée sur l'étude de ces variables principales : 
* 'nutriscore_grade' et 'nutrition-score-fr_100g'
* 'additives_n'
* 'allergens_n'
* 'nova_group'
* 'pnns_groups1' et 'pnns_groups_2'

Pour que ces colonnes soient bien renseignées, nous pouvons supprimer toutes les lignes ne contenant pas ces informations, ainsi que celles qui seront utilisées dans la suite du nettoyage ('code', 'url', 'product_name', 'brands_tags')

In [ ]:
data1 = data1.dropna(subset=['product_name', 'code', 'url', 'nutriscore_grade', 'nutrition-score-fr_100g', 'pnns_groups_1_bis', 'pnns_groups_2', 'brands', 'nova_group'], how = 'any')

In [ ]:
info_nettoyage(data1)

In [ ]:
graphe_donnees_manquantes(data1)

# Nettoyage des colonnes d'entiers

## *Nombre d'additifs*

In [ ]:
data1['additives_n'].unique()

In [ ]:
fig = plt.figure()
data1["additives_n"].hist()
plt.yscale('log')
ax = plt.axes()
ax = ax.set(xlabel='Nombre d\'additifs', ylabel='Nombre produits')
plt.title('Histogramme du nombre d\'additifs')
plt.show()

## *Nombre d'allergènes*

In [ ]:
data1['allergens'].unique()

In [ ]:
A = data1['allergens']
A = list(A.fillna('VM'))

In [ ]:
N =[]
for i in range(len(A)):
  if A[i] == 'VM':
    N.append(np.nan)
  else:
    N.append(A[i].count(',')+1)

data1['allergens_n'] = N

In [ ]:
data1[['allergens_n', 'allergens']].dropna()

In [ ]:
data1['allergens_n'] = data1['allergens_n'].astype('Int32')

In [ ]:
data1['allergens_n'].unique()

In [ ]:
fig = plt.figure()
data1["allergens_n"].hist()
ax = plt.axes()
ax.set_xlim([0, 25])
plt.yscale('log')
ax = ax.set(xlabel='Nombre d\'allergens', ylabel='Nombre produits')
plt.title('Histogramme du nombre d\'allergens')
plt.show()

In [ ]:
data1.drop(['allergens'], axis = 1, inplace = True)

## Energie des produits entre 'energy-kcal_100g' et 'energy_100g'

In [ ]:
nb_data_avant = data1.shape[0]

In [ ]:
var_energie = ['energy-kcal_100g', 'energy_100g']

Les aliments les plus caloriques tels que l'huile, le pain ou le beurre ne dépassent pas 900,00 kcal. On peut donc prendre comme limite pour l'énergie 1000 kcal, soit 4184kJ.

In [ ]:
data1 = data1[(np.invert(data1[['energy-kcal_100g']] > 1000)).all(axis=1)]
data1 = data1[(np.invert(data1[['energy_100g']] > 4500)).all(axis=1)]

In [ ]:
data1[var_energie].dropna().head(15)

In [ ]:
data1[var_energie].isnull().sum(axis = 0)

In [ ]:
tab = data1[var_energie].dropna(how = 'all')
tab[var_energie].isnull().sum(axis = 0)

La colonne 'energy_100g' étant la plus remplie et n'ayant pas de données manquantes qui seraient présentes dans la colonne'energy-kcal_100g', on peut utiliser celle-ci pour l'application en vérifiant si elle contient des erreurs grâce à la colonne 'energy-kcal_100g'. 

La relation entre 'energy_100g' et 'energy-kcal_100g' est : 1 kcal = 4,1868 kJ

In [ ]:
data1['coeff_proport'] = data1['energy_100g'].astype(float)/data1['energy-kcal_100g'].astype(float)

In [ ]:
data1[['energy_100g','energy-kcal_100g', 'coeff_proport']].describe(include='all')

In [ ]:
data1 = data1[(np.invert(data1[['coeff_proport']] == np.inf)).all(axis=1)] 
## correspond au quotient entre 'energy_100g' > 0 et 'energy-kcal_100g' = 0 (soit n'ayant pas un coefficient de proportionnalité proche 4.18)

In [ ]:
data1[['energy_100g','energy-kcal_100g', 'coeff_proport']].describe(include='all')

In [ ]:
## Pseudo-nuage de points
sns.set_theme(style="whitegrid")
sns.stripplot(x = "coeff_proport", data = data1, jitter = True)

Le coefficient de proportionnalité calculé entre 'energy_100g' et 'energy-kcal_100g' doit être de 4.184. Les outliers des coefficients correspondent donc à des erreurs dans le remplissage d'une ou des 2 colonnes.

In [ ]:
data1 = data1[(np.invert(data1[['coeff_proport']] < 4)).all(axis=1)]
data1 = data1[(np.invert(data1[['coeff_proport']] > 4.5)).all(axis=1)]

In [ ]:
## Pseudo-nuage de points
sns.set_theme(style="whitegrid")
sns.stripplot(x = "coeff_proport", data = data1, jitter = True)
plt.title('Pseudo-nuage de points')

In [ ]:
sns.boxplot(x=data1['coeff_proport'])

In [ ]:
data1[['energy_100g','energy-kcal_100g', 'coeff_proport']].describe(include='all')

In [ ]:
plt.figure(1,figsize=(10,5))
plt.gcf().subplots_adjust(left = 0.125, bottom = 0.2, right = 1.5,
                          top = 0.9, wspace = 0.5, hspace = 0.6)

plt.subplot(1,2,1)
sns.boxplot(x=data1["energy-kcal_100g"])
plt.title('Boxplot de l\'énergie en kcal')

plt.subplot(1,2,2)
sns.boxplot(x=data1["energy_100g"])
plt.title('Boxplot de l\'énergie en kJ')

plt.show()

In [ ]:
data1.drop(['energy-kcal_100g', 'coeff_proport'], axis = 1, inplace = True)

In [ ]:
data1 = data1.dropna(how = 'all')

In [ ]:
info_nettoyage(data1)

# Nettoyage des colonnes catégorielles

## Nova group

In [ ]:
data1['nova_group'].unique()

In [ ]:
data1['nova_group'] = data1['nova_group'].astype(CategoricalDtype(ordered=True))

In [ ]:
fig = plt.figure()
data1["nova_group"].hist()
ax = plt.axes()
ax = ax.set(xlabel='Nova group', ylabel='Nombre produits')
plt.title('Histogramme du Nova group')
plt.show()

## Nutriscore grade

In [ ]:
data1['nutriscore_grade'].unique()

In [ ]:
data1['nutriscore_grade'] = data1['nutriscore_grade'].astype(CategoricalDtype(categories=['a', 'b', 'c', 'd', 'e'], ordered=True))

In [ ]:
sns.set(style="white", rc={"axes.grid":True})
ax = sns.countplot(x='nutriscore_grade', data=data1, saturation = 1)
ax.set_title('Histogramme du Nutriscore grade')
ax = ax.set(xlabel='Nutriscore grade', ylabel='Nombre produits')

## Nettoyage entre 'nutriscore_grade' et 'nutrition-score-fr_100g'

In [ ]:
nb_data_avant = data1.shape[0]

In [ ]:
data1[['nutriscore_grade', 'nutrition-score-fr_100g']].describe(include='all')

In [ ]:
data1['indice_boissons'] = np.where(data1['pnns_groups_1_bis'] == 'Beverages', 'boissons', 'autres')

In [ ]:
pd.pivot_table(data=data1[['nutrition-score-fr_100g', 'nutriscore_grade','indice_boissons']],index=['indice_boissons', 'nutriscore_grade'])

In [ ]:
pd.pivot_table(data1[['nutrition-score-fr_100g', 'nutriscore_grade','indice_boissons']], index=['indice_boissons', 'nutriscore_grade'], values='nutrition-score-fr_100g', aggfunc=[np.min, np.max])

D'après le calcul du Nutriscore (https://fr.openfoodfacts.org/nutriscore), les relations qu'il y a entre le score de nutrition et le grade du nutriscore sont :

Pour les produits hors boissons, 

* un score compris entre -15 (min) et -1 correspond à un grade A
* un score compris entre 0 et 2 correspond à un grade B
* un score compris entre 3 et 10 correspond à un grade C
* un score compris entre 11 et 18 correspond à un grade D
* un score compris entre 19 et 40 (max) correspond à un grade E

Pour les boissons, 

* les eaux correspondent à un grade A
* un score compris entre -15 (min) et 1 correspond à un grade B
* un score compris entre 2 et 5 correspond à un grade C
* un score compris entre 6 et 9 correspond à un grade D
* un score compris entre 10 et 40 (max) correspond à un grade E

On peut vérifier si les variables 'nutriscore_grade' et 'nutrition-score-fr_100g' concordent entre elles selon ces relations en supprimant les lignes comportant des erreurs.


In [ ]:
## Pour les autres produits
data1 = data1.drop(data1[(data1['indice_boissons'] == 'autres') & (data1['nutriscore_grade'] == 'a') & (data1['nutrition-score-fr_100g'] > -1)].index)
data1 = data1.drop(data1[(data1['indice_boissons'] == 'autres') & (data1['nutriscore_grade'] == 'b') & (data1['nutrition-score-fr_100g'] < 0)].index)
data1 = data1.drop(data1[(data1['indice_boissons'] == 'autres') & (data1['nutriscore_grade'] == 'c') & (data1['nutrition-score-fr_100g'] < 3)].index)
data1 = data1.drop(data1[(data1['indice_boissons'] == 'autres') & (data1['nutriscore_grade'] == 'd') & (data1['nutrition-score-fr_100g'] < 11)].index)
data1 = data1.drop(data1[(data1['indice_boissons'] == 'autres') & (data1['nutriscore_grade'] == 'e') & (data1['nutrition-score-fr_100g'] < 19)].index)

In [ ]:
## Pour les boissons
data1 = data1.drop(data1[(data1['pnns_groups_2'] == 'Waters and flavored waters') & (data1['nutriscore_grade'] != 'a')].index)
data1 = data1.drop(data1[(data1['indice_boissons'] == 'boissons') & (data1['nutriscore_grade'] == 'b') & (data1['nutrition-score-fr_100g'] > 1)].index)
data1 = data1.drop(data1[(data1['indice_boissons'] == 'boissons') & (data1['nutriscore_grade'] == 'c') & (data1['nutrition-score-fr_100g'] > 5)].index)
data1 = data1.drop(data1[(data1['indice_boissons'] == 'boissons') & (data1['nutriscore_grade'] == 'd') & (data1['nutrition-score-fr_100g'] > 9)].index)
data1 = data1.drop(data1[(data1['indice_boissons'] == 'boissons') & (data1['nutriscore_grade'] == 'e') & (data1['nutrition-score-fr_100g'] < 19)].index)

In [ ]:
sns.boxplot(x="nutriscore_grade", y="nutrition-score-fr_100g", data=data1[(data1['indice_boissons'] == 'boissons')])
plt.title('Boxplot du score de nutrition en fonction nutriscore grade pour les Boissons')

In [ ]:
sns.boxplot(x="nutriscore_grade", y="nutrition-score-fr_100g", data=data1[(data1['indice_boissons'] == 'autres')])
plt.title('Boxplot du score de nutrition en fonction nutriscore grade pour les produits hors Boisons')

In [ ]:
data1 = data1.dropna(how = 'all')

In [ ]:
info_nettoyage(data1)

# Sélection des données en fonction de leur taux de remplissage

In [ ]:
data1 = data1.dropna(how = 'all')
data2 = data1.copy(deep=True)

In [ ]:
## Supprimer colonnes vides

cols_vides = [col for col in data2.columns if data2[col].isnull().all()]
cols_vides

data2.drop(cols_vides,axis=1,inplace=True)

In [ ]:
## Tableau du pourcentage de données présentes par colonne

X = pd.DataFrame(100-data2.isnull().sum(axis = 0)/len(data2)*100, columns = ["Pourcentage"])
X = X.sort_values(by = 'Pourcentage', ascending = False)
X

In [ ]:
## Histogramme du nombre de données présentes par colonne

fig, ax = plt.subplots(figsize=(12,56))
X['Pourcentage'].plot.barh(ax=ax, color='red')
plt.gca().invert_yaxis()
ax.set_title("Pourcentage de données présentes par colonne", fontweight='bold')
ax.xaxis.tick_top()
for i, v in enumerate(X['Pourcentage'].astype(int)):
    ax.text(v + 3, i + .25, str(v)+'%', color='red', fontweight='bold')

In [ ]:
graphe_donnees_manquantes(data2) 

Les données sont encore très éparses avec des colonnes quasiment vides. Le dataset est très vide et peu renseigné avec 66.7% de données manquantes. Comme 1er tri des données, on peut supprimer toutes les colonnes composées de moins de **X%** de données présentes. Ces variables ne peuvent être exploitées dans l'analyse au vu de la pauvreté des informations présentes.

## Choix du seuil X

In [ ]:
## Tableau du nombre de colonnes par intervalle de pourcentage de données présentes

X_intervalles = ['[0-10[', '[10-20[', '[20-30[', '[30-40[', '[40-50[', '[50-60[', '[60-70[', '[70-80[', '[80-90[', '[90-100]']
Y = [len(X[(X['Pourcentage']<10)])]

def condition(X, inf, sup):
  return len(X[(X['Pourcentage']>=inf) & (X['Pourcentage']<sup)])

for a in range(10, 90, 10):
  b=a+10
  Y.append(condition(X, a, b))

Y.append(len(X[(X['Pourcentage']>=90)]))

df = pd.DataFrame({'Intervalle des pourcentages de données présentes': X_intervalles, 'Nombres de colonnes': Y})
df

In [ ]:
## Histogramme pour choisir le seuil X

Y1 = list(reversed(Y))
nombre_colonnes_y = np.cumsum(Y1)
pourcentage_x = ['>90', '>80', '>70', '>60', '>50', '>40', '>30', '>20', '>10', '>0']

fig, ax = plt.subplots(1, figsize=(8, 5))
width = 0.75

ind = np.arange(len(nombre_colonnes_y))
ax.bar(ind, nombre_colonnes_y, width, color="red", align="center")

ax.set_xticks(ind+width/10)
ax.set_xticklabels(pourcentage_x, minor=False)

for i, v in enumerate(nombre_colonnes_y):
  ax.text((i-1)+0.8, nombre_colonnes_y[i]+1.5, str(v), color='black', fontweight='bold')
  ax.text((i-1)+0.8, nombre_colonnes_y[i]+10, str(str(int(v/144*100))+'%'), color='black', fontweight='bold')

ax.set(xlabel='Pourcentage minimum de données présentes dans une colonne', ylabel='Nombre de colonnes sur 144')
plt.title('Histogramme du choix du seuil')

plt.show()

In [ ]:
print("En moyenne, les colonnes ont", int(X['Pourcentage'].mean()), "% de données présentes, plus ou moins", int(X['Pourcentage'].std(ddof=0)), "%")

On obtient sur les 184 colonnes présentes dans le DataFrame :
* 152 colonnes (ie 100% des colonnes) ont plus de 0% de données présentes (ce qui est normal)
* 67 colonnes (ie 46% des colonnes) ont plus de 10% de données présentes
* 63 colonnes (ie 43% des colonnes) ont plus de 20% de données présentes
* 59 colonnes (ie 40% des colonnes) ont plus de 30% de données présentes

Grâce à ce graphique, on peut prendre la décision d'un seuil X de tri pour éliminer les colonnes contenant trop de données manquantes. 

On prend **X = 10%**

In [ ]:
## Suppression des colonnes
seuil = 10

colonnes_a_supprimer = list(X[X["Pourcentage"] < seuil].index)

data2 = data2.drop(columns=colonnes_a_supprimer)
print(len(colonnes_a_supprimer), "colonnes ont été supprimées.")

data2 = data2.dropna(how = 'all')

In [ ]:
info_nettoyage(data2)

In [ ]:
## Graphe des données manquantes du nouveau DataFrame data3
graphe_donnees_manquantes(data2)

Avec un seuil de 90%, on a un DataFrame avec : 
* 85 colonnes supprimées par rapport au DataFrame initial
* 25.1% de données manquantes
* 67 colonnes restantes par rapport au DataFrame initial

# Sélection des variables

In [ ]:
print(data2['brands'].isnull().sum(axis = 0))

In [ ]:
nb_data_avant = data2.shape[0]

In [ ]:
list(data2.columns)

La liste initiale des variables utiles a été modifiée

In [ ]:
colonnes_utiles = ['brands_tags','code', 'url', 'last_modified_t', 'product_name', 'additives_n', 'allergens_n','nutriscore_grade', 'nova_group', 'pnns_groups_1_bis', 'pnns_groups_2'] + [x for x in list(data2.columns)  if (x.endswith('_100g'))]
colonnes_utiles

In [ ]:
data3 = data2[colonnes_utiles]
data3 = data3.dropna(how = 'all')

In [ ]:
## Graphe des données manquantes du nouveau DataFrame data4
graphe_donnees_manquantes(data3)

In [ ]:
info_nettoyage(data3)

# Doublons

In [ ]:
nb_data_avant = data3.shape[0]

In [ ]:
data3['product_name'] = data3['product_name'].str.lower()
data3['product_name'] = data3['product_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
data3['product_name'] = data3['product_name'].astype(str)
data3['taille_nom_product'] = data3['product_name'].map(len)
data3 = data3[(np.invert(data3[['taille_nom_product']] <= 1)).all(axis=1)]

In [ ]:
data3[['code', 'url', 'product_name']].head(8)

In [ ]:
data4 = data3.copy(deep=True)
nb_data_avant = data4.shape[0]

In [ ]:
data4['brands_tags'] = data4['brands_tags'].str.lower()
data4['brands_tags'] = data4['brands_tags'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
data4['brands_tags'].describe(include='all')

In [ ]:
## Supprimer les doublons en fonction du code-barre en gardant la ligne du produit la plus récente
data4 = data4.sort_values('last_modified_t', ascending=False).drop_duplicates('code', keep='first').sort_index().reset_index(drop=True)

In [ ]:
## Supprimer les doublons en fonction de l'url en gardant la ligne du produit la plus récente
data4 = data4.sort_values('last_modified_t', ascending=False).drop_duplicates('url', keep='first').sort_index().reset_index(drop=True)

In [ ]:
## Supprimer les doublons en fonction du nom du produit en gardant la ligne du produit la plus récente
data4 = data4.sort_values('last_modified_t', ascending=False).drop_duplicates(subset=['product_name', 'brands_tags'], keep='first').sort_index().reset_index(drop=True)

In [ ]:
info_nettoyage(data4)

In [ ]:
data4.drop(['url', 'last_modified_t', 'taille_nom_product', 'brands_tags'], axis = 1, inplace = True)

# Remplacer les cases vides des nutriments, des additifs et des allergènes par 0

In [ ]:
nb_data_avant = data4.shape[0]

In [ ]:
print(data4.columns)

On fait l'hypothèse que les données manquantes des variables des nutriments, du nombre d'additifs et du nombre d'allergènes ne correspondent pas à des données non renseignées mais à l'absence d'additif, d'allergène et de nutriment dans le produit.

On se place dans le cadre de cette hypothèse en remplaçant toutes les données manquantes pour ces variables par 0.

In [ ]:
colonnes_0 = ['allergens_n', 'additives_n']  + [x for x in list(data4.columns) if (x.endswith('_100g') and x not in (['energy_100g', 'nutrition-score-fr_100g']))]
colonnes_0

In [ ]:
data5 = data4.copy(deep=True)

In [ ]:
data5[colonnes_0].describe(include = 'all')

In [ ]:
for i in range(len(colonnes_0)):
  data5[colonnes_0[i]] = data5[colonnes_0[i]].replace(np.nan, 0)

In [ ]:
print(data5[colonnes_0].isnull().sum(axis = 0))

In [ ]:
data5[colonnes_0].describe(include = 'all')

In [ ]:
## Graphe des données manquantes du nouveau DataFrame data3
graphe_donnees_manquantes(data5)

In [ ]:
info_nettoyage(data5)

# Outliers

In [ ]:
print(data5.columns)

## Variables qualitatives

In [ ]:
VAR_quali = ['nutriscore_grade', 'nova_group', 'pnns_groups_1_bis', 'pnns_groups_2']

In [ ]:
data5[VAR_quali] = data5[VAR_quali].astype('category')

In [ ]:
data5[VAR_quali].describe(include='all')

## Variables quantitatives

In [ ]:
# VAR_numeric = data3.select_dtypes(include=[np.number])
VAR_numeric = ['additives_n', 'allergens_n', 'energy_100g','nutrition-score-fr_100g',
               'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g','fiber_100g', 'proteins_100g', 'salt_100g', 'sodium_100g']

In [ ]:
sns.distplot(data5['additives_n'], bins = 6, kde = False)
plt.yscale('log')

In [ ]:
sns.distplot(data5['allergens_n'], bins = 6, kde = False)
plt.yscale('log')

In [ ]:
sns.set_theme(style="whitegrid")
sns.boxplot(x=data5['energy_100g'])

In [ ]:
info_nutriments = VAR_numeric[4:11]

In [ ]:
plt.figure(figsize = [15,7])
sns.boxplot(x="variable", y="value", data=data5[info_nutriments].melt())
plt.title("Boxplot des variables nutritionnelles")
plt.xticks(rotation = 90);
plt.show()

In [ ]:
data5[info_nutriments].describe(include='all')

# Exportation du jeu de données final

In [ ]:
nb_data_avant = data5.shape[0]

In [ ]:
data5.info()

In [ ]:
print("nombre de dimensions de data: ", data5.ndim)
print("forme de data: ", data5.shape)
print("taille de data: ", data5.size)

In [ ]:
data5.head(5)

In [ ]:
graphe_donnees_manquantes(data5) 

In [ ]:
info_nettoyage(data5)

In [ ]:
# data5.to_csv('clean_nutri_data.csv', encoding='utf_8', index=False)
data5.to_csv('en.openfoodfacts.org.products_clean.csv', sep = '\t')
files.download('en.openfoodfacts.org.products_clean.csv')